In [15]:
from fredapi import Fred
import pandas as pd
import numpy as np
from datetime import timedelta
import scipy

In [2]:
with open("../API_KEY.txt") as txt:
    API_KEY = txt.read()

fred = Fred(api_key=API_KEY)

In [3]:
wilshire_search = fred.search('Wilshire 5000', limit=5)
wilshire_search.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
WILL5000INDFC,WILL5000INDFC,2023-02-22,2023-02-22,Wilshire 5000 Total Market Full Cap Index,1970-12-31,2023-02-21,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-22 07:04:05-06:00,67,The observations for the Wilshire 5000 Total M...
WILL5000PRFC,WILL5000PRFC,2023-02-22,2023-02-22,Wilshire 5000 Full Cap Price Index,1970-12-31,2023-02-21,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-22 07:04:05-06:00,56,The observations for the Wilshire 5000 Full Ca...
WILL5000PR,WILL5000PR,2023-02-22,2023-02-22,Wilshire 5000 Price Index,1970-12-31,2023-02-21,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-22 07:04:05-06:00,54,The observations for the Wilshire 5000 Price I...
WILL5000IND,WILL5000IND,2023-02-22,2023-02-22,Wilshire 5000 Total Market Index,1970-12-31,2023-02-21,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-22 07:04:03-06:00,47,The observations for the Wilshire 5000 Total M...


In [4]:

start_date = "1979-12-31"
end_date = "2017-12-31"
will5000 = fred.get_series(series_id='WILL5000IND', observation_start=start_date, observation_end=end_date)
will5000 = will5000.to_frame()

In [5]:
will5000.isna().sum()
will5000 = will5000.dropna()
will5000 = will5000.reset_index()
will5000 = will5000.rename(columns={"index": "date", 0: "TR"})
will5000

,date,TR
0,1979-12-31,1.90
1,1980-01-02,1.86
2,1980-01-04,1.88
3,1980-01-07,1.89
4,1980-01-08,1.93
...,...,...
9579,2017-12-22,124.03
9580,2017-12-26,123.94
9581,2017-12-27,124.04
9582,2017-12-28,124.33


In [6]:
will5000['log_ret'] = np.log(will5000.TR/will5000.TR.shift(1))
will5000['dis_ret'] = will5000.TR.pct_change()
#will5000['dis_ret'] = np.exp(will5000.log_ret)-1

will5000 = will5000.dropna()

In [7]:
print(will5000.head(5))
print("\n###########################################\n")
print(will5000.tail(5))

        date    TR   log_ret   dis_ret
1 1980-01-02  1.86 -0.021277 -0.021053
2 1980-01-04  1.88  0.010695  0.010753
3 1980-01-07  1.89  0.005305  0.005319
4 1980-01-08  1.93  0.020943  0.021164
5 1980-01-09  1.93  0.000000  0.000000

###########################################

           date      TR   log_ret   dis_ret
9579 2017-12-22  124.03 -0.000564 -0.000564
9580 2017-12-26  123.94 -0.000726 -0.000726
9581 2017-12-27  124.04  0.000807  0.000807
9582 2017-12-28  124.33  0.002335  0.002338
9583 2017-12-29  123.67 -0.005323 -0.005308


In [8]:
df = will5000.copy()

df_w = df.groupby([pd.Grouper(key="date", freq="W")])["log_ret"].sum().to_frame().reset_index()
df_w["date"] = pd.to_datetime(df_w.date) - timedelta(days=2)
df_w["dis_ret"] = np.exp(df_w.log_ret)-1

df_m = df.groupby([pd.Grouper(key="date", freq="M")])["log_ret"].sum().to_frame().reset_index()
df_m["dis_ret"] = np.exp(df_m.log_ret)-1

df_q = df.groupby([pd.Grouper(key="date", freq="Q")])["log_ret"].sum().to_frame().reset_index()
df_q["dis_ret"] = np.exp(df_q.log_ret)-1

df_y = df.groupby([pd.Grouper(key="date", freq="Y")])["log_ret"].sum().to_frame().reset_index()
df_y["dis_ret"] = np.exp(df_y.log_ret)-1


In [22]:

mu = will5000["log_ret"].mean() 
sig = will5000["log_ret"].std() 
print(round(mu, 8))
print(round(sig, 8))

value_at_risk = scipy.stats.norm.ppf(0.05, loc=mu, scale=sig)
print(round(value_at_risk, 6))

HFvar = 1000 * (np.exp(value_at_risk)-1) 
print(f"{round(HFvar, 1)} Mil loss on a single day in a 1B portfolio")
'''
es = mu-sig*dnorm(qnorm(0.05, 0, 1), 0, 1)/0.05
round(es, 6)
round((1000*(exp(es)-1)), 1) 
'''


0.00043575
0.01072056
-0.017198
-17.1 Mil loss on a single day in a 1B portfolio


'\nes = mu-sig*dnorm(qnorm(0.05, 0, 1), 0, 1)/0.05\nround(es, 6)\nround((1000*(exp(es)-1)), 1) \n'

In [ ]:
'''The equivalent of the R pnorm() 
function is: scipy.stats.norm.cdf()

with python The equivalent of the R qnorm()
function is: scipy.stats.norm.ppf()
with python
'''